<h3>This Notebook will contain some of the final steps taken with the ACS Income Dataset<h3>
<h4>There are a number of steps that I will be taking here to see the solution across multiple samples of ACSincome data<h4>
<ul>
<li>Take 5 samples of the ACSIncome data: 1k, 5k, 10k, 15k, 20k</li>
<li>Flip 20% of the datapoints rather than 50%</li>
<li>Represent the changes as percentages rather than individual datapoints</li>
<li>This file will also contain all the sensitive attributes in order see them against each other</li>
</ul>

In [1]:
#Import all of the necessary libraries in
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from folktables import ACSDataSource, ACSIncome
import math
warnings.filterwarnings('ignore')

In [ ]:
#This function is being used to calculate the statistical parity of the data set
#test_data: The original data frame containing the test data
#predictors: List of the all the column names AKA IV's
#prediction_df: Contains the predicitons made by the DT model
#sensitive_attr: Name of the sensitive attr
#sensitive_attr_cutoff: Cut off value for sensitive attr
#concat_col: Name of column I am creating new dataframe on

def s_parity(test_data, predictors, prediction_df, sensitive_attr, concat_col):
    #Creating a new DF that contains all the datapoints from the test data and the predictions made from LR model
    #Concat_col: outcome
    test_demo_df = pd.DataFrame(test_data, columns = predictors)
    predicted_df = pd.DataFrame(prediction_df, columns = [concat_col])
    concat_df = pd.concat([test_demo_df,predicted_df], axis=1)

    #Get the two groups of people totals
    total_unpriv = (len(concat_df[concat_df[sensitive_attr]==0]))
    total_priv = (len(concat_df[concat_df[sensitive_attr]==1]))

    #Number of people accepted
    total_credit_unpriv = len(concat_df[(concat_df[concat_col] == 1) & (concat_df[sensitive_attr] == 0)])
    total_credit_priv = len(concat_df[(concat_df[concat_col] == 1) & (concat_df[sensitive_attr] == 1)])

    #Percentage of approved people
    p_unpriv = total_credit_unpriv/total_unpriv
    p_priv = total_credit_priv/total_priv


    #Calculate the parity
    parity = p_priv - p_unpriv


    return parity

In [ ]:
#Make changes to make more efficient
#Function used to run the DT model
#train_dataset: Training dataset to train the model
#independent_var: Column names
#dependent_var: Prediction column name
#concat_col: Name of column creating new DF on
def logistic_regression(train_dataset, independent_var, dependent_var, concat_col):
        #Split the data up into train and test values and then run the DT model
        #These steps aren't neccessary to consistenly do over and over again
        x = train_dataset[independent_var].values
        y = train_dataset[dependent_var].values
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=310)
        clf = LogisticRegression(class_weight=None, max_iter=100)
        log = clf.fit(x_train,y_train)
        prediction = log.predict(x_test)

        #Printing the Overall accuracy of the model after one run
        #F1_Score=metrics.f1_score(y_test, prediction, average='weighted')
        #file.write(f'\nAccuracy of the model on Testing Sample Data: {F1_Score}')

        #Prints out the average across all ten run throughs
        #Accuracy_Values=cross_val_score(log, x , y, cv=10, scoring='f1_weighted')

        accuracy = accuracy_score(y_test,prediction)*100

        #After running the model I return a df with the datapoints and the labels
        test_demo_df = pd.DataFrame(x_test, columns = independent_var)
        #Concat_col: credit_risk_12
        predicted_df = pd.DataFrame(prediction, columns = [concat_col])

        return accuracy, test_demo_df, predicted_df

<h5>Setup the data with the random samples from acs_income.csv<h5>

In [2]:
acs_dataframe = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',')

In [12]:
#Split the data into different dataframes to represent the samples

#1k datpoints
onek_datapoints = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',').sample(1000)
onek_datapoints=onek_datapoints.reset_index(drop=True)

#5k datapoints
fivek_datapoints = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',').sample(5000) 
fivek_datapoint = fivek_datapoints.reset_index(drop=True)

#10k datapoints
tenk_datapoints = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',').sample(10000) 
tenk_datapoint = tenk_datapoints.reset_index(drop=True)

#15k datapoints
fifteenk_datapoints = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',').sample(15000)
fifteenk_datapoints = fifteenk_datapoints.reset_index(drop=True)

#20k datapoints
twentyk_datapoints = pd.read_csv('acs_data/acs_income.csv', index_col=None, sep=',').sample(20000)
twentyk_datapoints = twentyk_datapoints.reset_index(drop=True)